In [5]:
import pandas as pd
import numpy as np
import librosa.core as lr
from librosa.display import specshow
from librosa import feature
from glob import glob
import os
from IPython.display import Audio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from zipfile import ZipFile
import urllib
from tqdm.notebook import tqdm

sounds_dir = "xeno-canto-ca-nv/"

def fullPath(sample): return os.path.join(sounds_dir, sample['file_name'])

df = pd.read_csv("xeno-canto_ca-nv_index.csv")
df.columns = np.append('id', df.columns[1:None])#label first col as id
files_list = glob(os.path.join(sounds_dir,"*.mp3"))

print("%i mp3 files in %s"%(len(files_list), sounds_dir))
print("%i samples in index."%len(df))
df = df.drop(['country', 'license', 'remarks', 'recordist', 'recordist_url', 'location', 'english_cname', 'file_url'], axis = 1)
df.head()

2730 mp3 files in xeno-canto-ca-nv/
2730 samples in index.


,id,duration_seconds,file_id,file_name,genus,latitude,longitude,sonogram_url,species,type,full_name
0,0,3,17804,XC17804.mp3,Melozone,33.3117,-114.68912,https://www.xeno-canto.org/sounds/uploaded/EKK...,aberti,'seet' call,Abert's Towhee (Melozone aberti)
1,1,4,177367,XC177367.mp3,Melozone,34.2850,-114.06900,https://www.xeno-canto.org/sounds/uploaded/YQN...,aberti,call,Abert's Towhee (Melozone aberti dumeticola)
2,2,4,145505,XC145505.mp3,Melozone,34.2850,-114.06900,https://www.xeno-canto.org/sounds/uploaded/YQN...,aberti,Squeal duet,Abert's Towhee (Melozone aberti dumeticola)
3,3,5,228159,XC228159.mp3,Melozone,33.1188,-115.79450,https://www.xeno-canto.org/sounds/uploaded/OOE...,aberti,interaction duet,Abert's Towhee (Melozone aberti)
4,4,5,51313,XC51313.mp3,Melozone,36.0628,-115.11280,https://www.xeno-canto.org/sounds/uploaded/PWD...,aberti,call,Abert's Towhee (Melozone aberti dumeticola)


In [2]:
# generate class feature for easy supervised learning in sklearn
makeClass = lambda a, label: 1 if a.lower().find(label.lower()) != -1 else 0
makeClass = np.vectorize(makeClass)

y = makeClass(df['type'].values, 'song') #class feature is song/notsong
df['class'] = y
df.head()

,id,duration_seconds,file_id,file_name,genus,latitude,longitude,sonogram_url,species,type,full_name,class
0,0,3,17804,XC17804.mp3,Melozone,33.3117,-114.68912,https://www.xeno-canto.org/sounds/uploaded/EKK...,aberti,'seet' call,Abert's Towhee (Melozone aberti),0
1,1,4,177367,XC177367.mp3,Melozone,34.2850,-114.06900,https://www.xeno-canto.org/sounds/uploaded/YQN...,aberti,call,Abert's Towhee (Melozone aberti dumeticola),0
2,2,4,145505,XC145505.mp3,Melozone,34.2850,-114.06900,https://www.xeno-canto.org/sounds/uploaded/YQN...,aberti,Squeal duet,Abert's Towhee (Melozone aberti dumeticola),0
3,3,5,228159,XC228159.mp3,Melozone,33.1188,-115.79450,https://www.xeno-canto.org/sounds/uploaded/OOE...,aberti,interaction duet,Abert's Towhee (Melozone aberti),0
4,4,5,51313,XC51313.mp3,Melozone,36.0628,-115.11280,https://www.xeno-canto.org/sounds/uploaded/PWD...,aberti,call,Abert's Towhee (Melozone aberti dumeticola),0


In [ ]:
#load sonograms into np array and prep for PCA for clustering
# sonograms = mpimg.imread(df['sonogram_url'][0]).ravel()
# print(sonograms)
# for i in tqdm(range(0,2730)):
#     temp = np.array(mpimg.imread(df['sonogram_url'][i])).ravel()
#     sonograms = np.vstack((sonograms, temp))

#this errors eventually, probably better to make them myself.

In [6]:
#get length for zero padding
#temp = mpimg.imread(df['sonogram_url'][0])
#print(np.array(temp).shape())

#for i in tqdm(range(0,2730)):
features = {}
for sample_idx, sample in tqdm(df.iterrows(), total=len(df)):
    data, samplerate = lr.load(fullPath(sample), 'mp3')
    features[sample['file_id']] = {
        'melspectrogram': lr.feature.melspectrogram(data, sr=samplerate),
        'mfcc': lr.feature.mfcc(data, sr=samplerate ),
        'spectral_centroid': lr.feature.spectral_centroid(data, sr=samplerate),
        'spectral_bandwidth': lr.feature.spectral_bandwidth(data, sr=samplerate),
    }

NoBackendError: 